In [1]:
import pandas as pd

In [2]:
posts = pd.read_parquet('items.parquet')
posts.head()

,authorId,content,itemId,title
0,22062724202903043,Британский кот — олицетворение невозмутимости....,1155255918526217609,"Как воспитать британского кота добрым, ласковы..."
1,5021973993129217102,Комнатные растения можно встретить в квартирах...,3466678402942852201,Вызывают судороги и смерть: пять опасных комна...
2,5843309594367696221,КОНТЕЙНЕР И БЫТОВКА Бывший в употреблении морс...,9802178366207388656,Из чего построить дачный дом? Семь доступных в...
3,4785891415385376217,Советский разведчик Аркадий Корнев с гранатой ...,5959787242217250844,Как осажденные советские солдаты в Афганистане...
4,6405657875686126019,Преимущества и недостатки оснастки и техники...,14212405135790401725,Преимущества и недостатки оснастки и техники д...


In [3]:
train = pd.read_csv('train.csv')
train.head()

,leftItemId,rightItemId,target
0,8456188648456666255,17842617327504664637,relevant_minus
1,2587705644018673303,4730919752548875048,relevant_minus
2,14440570074797857092,9007102992088629866,relevant
3,10768915202093280757,8382562539112058142,relevant
4,2799154957886243971,11333177839206244806,no_relevant


In [4]:
embeds = pd.read_parquet('embeds.parquet')
embeds.head()

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_303,emb_304,emb_305,emb_306,emb_307,emb_308,emb_309,emb_310,emb_311,itemId
0,0.121566,0.026545,0.004355,-0.020323,-0.021250,-0.029509,0.035062,-0.019077,0.001802,0.080969,...,0.021640,0.054264,0.109156,0.019327,0.024834,0.003783,0.046775,0.055629,-0.053721,1155255918526217609
1,0.103179,-0.012450,-0.002536,-0.032106,-0.014251,-0.018544,0.007134,-0.045286,-0.038251,0.057495,...,-0.008583,0.059182,-0.019912,-0.023639,-0.004447,-0.019848,0.032108,-0.008451,-0.000064,3466678402942852201
2,0.031214,0.033021,-0.024075,-0.023425,-0.027527,-0.001742,0.037039,-0.012754,0.009106,0.106402,...,-0.018187,0.026627,0.042350,-0.040521,0.025986,-0.023414,0.017387,0.010831,-0.048144,9802178366207388656
3,0.004758,-0.039083,-0.027797,-0.029786,-0.035008,0.002692,0.046693,-0.073959,-0.065016,0.127048,...,0.008112,0.028288,0.046319,0.001199,-0.017831,-0.077715,0.012925,0.091118,-0.066211,5959787242217250844
4,0.035374,0.008162,0.042682,-0.092922,0.034032,-0.008354,0.060384,0.013992,0.031133,0.102073,...,0.001804,0.015231,0.022245,0.009479,0.006322,-0.067421,0.039689,0.069862,-0.040986,14212405135790401725


In [5]:
train = (
    train
    .merge(posts, left_on='leftItemId', right_on='itemId')
    .merge(posts, left_on='rightItemId', right_on='itemId')
)

In [6]:
train = train.drop(columns=['itemId_x', 'itemId_y'])

In [7]:
train = (
    train
    .merge(embeds.add_prefix('left_'), left_on='leftItemId', right_on='left_itemId')
    .merge(embeds.add_prefix('right_'), left_on='rightItemId', right_on='right_itemId')
)

In [8]:
train = train[train['target'].isin(['relevant_plus', 'relevant', 'relevant_minus', 'no_relevant'])]

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
vec_cols_x = [c for c in train.columns if c.startswith('left_emb_')]
vec_cols_y = [c for c in train.columns if c.startswith('right_emb_')]

vecs_x = train[vec_cols_x].to_numpy()
vecs_y = train[vec_cols_y].to_numpy()

In [11]:
from tqdm import tqdm

In [12]:
# sims = list(sim for sim in tqdm(map(lambda x: cosine_similarity(x[0].reshape(1, -1), x[1].reshape(1, -1)), zip(vecs_x, vecs_y)), total=len(vecs_x)))

In [13]:
# train['sim'] = list(map(lambda x: x[0][0], sims))

In [14]:
train['sim'] = (vecs_x * vecs_y).sum(axis=1) / (
    (vecs_x**2).sum(axis=1)**0.5 * (vecs_y**2).sum(axis=1)**0.5
)

# предобаботка текста

In [ ]:
# import re
# import string
# import emoji

# tqdm.pandas()

# def clean_text(text):
#     text = text.lower()
#     text = emoji.replace_emoji(text, replace=' ')  # убрать эмодзи
#     text = re.sub(r'https?://\S+|www\.\S+', ' ', text)  # убрать ссылки
#     text = re.sub(r'@\w+|#\w+', ' ', text)  # убрать @user и #теги
#     text = re.sub(r'[^а-яa-z0-9\s' + string.punctuation + ']', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()
#     return text

# for col in ['title_x', 'content_x', 'title_y', 'content_y']:
#     train[col] = train[col].fillna('').progress_apply(clean_text)


  0%|          | 0/208494 [00:00<?, ?it/s]

100%|██████████| 208494/208494 [12:59<00:00, 267.63it/s]


In [16]:
import numpy as np
import re
import string
import emoji

def text_features(df, prefix, title_col, content_col):
    for col in [title_col, content_col]:
        name = f'{prefix}_{col.split("_")[-1]}'
        s = df[col].fillna("")

        # 1. Общие статистики
        df[f'{name}_len'] = s.str.len()
        df[f'{name}_words'] = s.apply(lambda x: len(x.split()))
        df[f'{name}_unique_words'] = s.apply(lambda x: len(set(x.split())))
        df[f'{name}_word_uniqueness'] = df[f'{name}_unique_words'] / (df[f'{name}_words'] + 1e-5)

        # 2. Символы и структура
        df[f'{name}_digits'] = s.apply(lambda x: sum(c.isdigit() for c in x))
        df[f'{name}_upper'] = s.apply(lambda x: sum(c.isupper() for c in x))
        df[f'{name}_punct'] = s.apply(lambda x: sum(c in string.punctuation for c in x))
        df[f'{name}_spaces'] = s.apply(lambda x: sum(c.isspace() for c in x))
        # df[f'{name}_emoji'] = s.apply(lambda x: len(emoji.emoji_list(x)))

        # 3. Отношения
        df[f'{name}_upper_ratio'] = df[f'{name}_upper'] / (df[f'{name}_len'] + 1e-5)
        df[f'{name}_punct_ratio'] = df[f'{name}_punct'] / (df[f'{name}_len'] + 1e-5)
        df[f'{name}_digit_ratio'] = df[f'{name}_digits'] / (df[f'{name}_len'] + 1e-5)
        # df[f'{name}_emoji_ratio'] = df[f'{name}_emoji'] / (df[f'{name}_len'] + 1e-5)

        # 4. Средние длины слов
        df[f'{name}_avg_word_len'] = s.apply(lambda x: np.mean([len(w) for w in x.split()]) if x.split() else 0)

        # 5. Повторы и разнообразие
        df[f'{name}_repeated_words'] = s.apply(
            lambda x: sum(v > 1 for v in pd.Series(x.split()).value_counts().values)
        )
        df[f'{name}_repeat_ratio'] = df[f'{name}_repeated_words'] / (df[f'{name}_words'] + 1e-5)

        # 6. Спецсимволы
        df[f'{name}_excl_marks'] = s.str.count('!')
        df[f'{name}_quest_marks'] = s.str.count(r'\?')
        df[f'{name}_dots'] = s.str.count(r'\.')
        df[f'{name}_commas'] = s.str.count(',')
        df[f'{name}_quotes'] = s.str.count(r'["«»“”]')
        df[f'{name}_brackets'] = s.str.count(r'[\(\)\[\]\{\}]')

        # 7. Отношения пунктуации
        df[f'{name}_exc_ratio'] = df[f'{name}_excl_marks'] / (df[f'{name}_len'] + 1e-5)
        df[f'{name}_quest_ratio'] = df[f'{name}_quest_marks'] / (df[f'{name}_len'] + 1e-5)

        # # 8. Лингвистические паттерны
        # df[f'{name}_avg_sent_len'] = s.apply(lambda x: np.mean([len(sent.split()) for sent in re.split(r'[.!?]', x) if sent.strip()]) if x else 0)
        # df[f'{name}_num_sentences'] = s.apply(lambda x: len([sent for sent in re.split(r'[.!?]', x) if sent.strip()]))
        # df[f'{name}_has_ellipsis'] = s.str.contains(r'\.\.\.').astype(int)
        # df[f'{name}_has_mention'] = s.str.contains(r'@\w+').astype(int)
        # df[f'{name}_has_url'] = s.str.contains(r'https?://|www\.').astype(int)
        # df[f'{name}_has_hashtag'] = s.str.contains(r'#\w+').astype(int)

        # 9. Сложность текста
        df[f'{name}_lexical_richness'] = df[f'{name}_unique_words'] / (df[f'{name}_words'] + 1)
        df[f'{name}_avg_word_complexity'] = s.apply(
            lambda x: np.mean([sum(c in "аеёиоуыэюяaeiouy" for c in w.lower()) / (len(w)+1e-5) for w in x.split()]) if x.split() else 0
        )

text_features(train, 'left', 'title_x', 'content_x')
print('готово1')
text_features(train, 'right', 'title_y', 'content_y')
print('готово2')

готово1
готово2


# catboost

In [17]:
from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import train_test_split

In [18]:
X = train.drop(columns=['leftItemId', 'rightItemId', 'target', 'authorId_x', 'authorId_y', 'right_itemId', 'left_itemId'])
y = train['target']

text_features = ['title_x', 'title_y', 'content_x', 'content_y']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=185)

In [20]:
train_pool = Pool(
    data=X_train,
    label=y_train,
    text_features=text_features
)

test_pool = Pool(
    data=X_test,
    label=y_test,
    text_features=text_features
)

In [21]:
params = dict(
    task_type='GPU',
    eval_metric='TotalF1:average=Weighted',
    loss_function='MultiClass',
    iterations=8000,
    learning_rate=0.03,
    depth=8,
    border_count=254,
    l2_leaf_reg=10,
    random_seed=185,
    od_type='Iter',
    od_wait=100,
    use_best_model=True,
    verbose=50,
    text_features=text_features
)

In [47]:
model = CatBoostClassifier(iterations=3000, learning_rate=0.08, verbose=50, task_type='GPU', border_count=254, 
    eval_metric='TotalF1:average=Weighted',)

In [48]:
model.fit(train_pool
          , eval_set=test_pool
          )

0:	learn: 0.4396863	test: 0.4448038	best: 0.4448038 (0)	total: 316ms	remaining: 15m 46s
50:	learn: 0.4765181	test: 0.4822717	best: 0.4823242 (48)	total: 4.83s	remaining: 4m 39s
100:	learn: 0.4862425	test: 0.4885938	best: 0.4885938 (100)	total: 9.51s	remaining: 4m 32s
150:	learn: 0.4937343	test: 0.4913733	best: 0.4914035 (148)	total: 13.9s	remaining: 4m 23s
200:	learn: 0.4987947	test: 0.4936606	best: 0.4940374 (188)	total: 18.2s	remaining: 4m 14s
250:	learn: 0.5029632	test: 0.4958213	best: 0.4959515 (248)	total: 22.6s	remaining: 4m 7s
300:	learn: 0.5071741	test: 0.4977197	best: 0.4979121 (298)	total: 26.9s	remaining: 4m
350:	learn: 0.5112803	test: 0.4981111	best: 0.4987777 (341)	total: 31.1s	remaining: 3m 54s
400:	learn: 0.5151385	test: 0.4992725	best: 0.4995730 (392)	total: 35.4s	remaining: 3m 49s
450:	learn: 0.5189098	test: 0.4997194	best: 0.4997194 (450)	total: 39.7s	remaining: 3m 44s
500:	learn: 0.5224106	test: 0.4987112	best: 0.4998206 (467)	total: 43.9s	remaining: 3m 39s
550:	lear

In [49]:
from sklearn.metrics import f1_score

In [50]:
preds_val = model.predict(test_pool)

In [51]:
f1_score(y_test, preds_val, average='weighted')

0.505710735286079

# inference for test

In [27]:
test = pd.read_csv('test.csv')
test.head()

,leftItemId,rightItemId
0,15923600479346774213,10591191871885021601
1,7866768712250438268,9584895322191308561
2,6742770626780529612,6966853603787522235
3,4113267524593114411,16868459055811063062
4,18183962070656736836,17204657866960404813


In [28]:
test = test.merge(posts, left_on='leftItemId', right_on='itemId').merge(posts, left_on='rightItemId', right_on='itemId')

In [29]:
test = test.drop(columns=['itemId_x', 'itemId_y'])
test = (
    test
    .merge(embeds.add_prefix('left_'), left_on='leftItemId', right_on='left_itemId')
    .merge(embeds.add_prefix('right_'), left_on='rightItemId', right_on='right_itemId')
)

In [30]:
vec_cols_x = [c for c in test.columns if c.startswith('left_emb_')]
vec_cols_y = [c for c in test.columns if c.startswith('right_emb_')]

vecs_x = test[vec_cols_x].to_numpy()
vecs_y = test[vec_cols_y].to_numpy()

In [31]:
test['sim'] = (vecs_x * vecs_y).sum(axis=1) / (
    (vecs_x**2).sum(axis=1)**0.5 * (vecs_y**2).sum(axis=1)**0.5
)

In [32]:
import numpy as np
import re
import string
import emoji

def text_features(df, prefix, title_col, content_col):
    for col in [title_col, content_col]:
        name = f'{prefix}_{col.split("_")[-1]}'
        s = df[col].fillna("")

        # 1. Общие статистики
        df[f'{name}_len'] = s.str.len()
        df[f'{name}_words'] = s.apply(lambda x: len(x.split()))
        df[f'{name}_unique_words'] = s.apply(lambda x: len(set(x.split())))
        df[f'{name}_word_uniqueness'] = df[f'{name}_unique_words'] / (df[f'{name}_words'] + 1e-5)

        # 2. Символы и структура
        df[f'{name}_digits'] = s.apply(lambda x: sum(c.isdigit() for c in x))
        df[f'{name}_upper'] = s.apply(lambda x: sum(c.isupper() for c in x))
        df[f'{name}_punct'] = s.apply(lambda x: sum(c in string.punctuation for c in x))
        df[f'{name}_spaces'] = s.apply(lambda x: sum(c.isspace() for c in x))
        # df[f'{name}_emoji'] = s.apply(lambda x: len(emoji.emoji_list(x)))

        # 3. Отношения
        df[f'{name}_upper_ratio'] = df[f'{name}_upper'] / (df[f'{name}_len'] + 1e-5)
        df[f'{name}_punct_ratio'] = df[f'{name}_punct'] / (df[f'{name}_len'] + 1e-5)
        df[f'{name}_digit_ratio'] = df[f'{name}_digits'] / (df[f'{name}_len'] + 1e-5)
        # df[f'{name}_emoji_ratio'] = df[f'{name}_emoji'] / (df[f'{name}_len'] + 1e-5)

        # 4. Средние длины слов
        df[f'{name}_avg_word_len'] = s.apply(lambda x: np.mean([len(w) for w in x.split()]) if x.split() else 0)

        # 5. Повторы и разнообразие
        df[f'{name}_repeated_words'] = s.apply(
            lambda x: sum(v > 1 for v in pd.Series(x.split()).value_counts().values)
        )
        df[f'{name}_repeat_ratio'] = df[f'{name}_repeated_words'] / (df[f'{name}_words'] + 1e-5)

        # 6. Спецсимволы
        df[f'{name}_excl_marks'] = s.str.count('!')
        df[f'{name}_quest_marks'] = s.str.count(r'\?')
        df[f'{name}_dots'] = s.str.count(r'\.')
        df[f'{name}_commas'] = s.str.count(',')
        df[f'{name}_quotes'] = s.str.count(r'["«»“”]')
        df[f'{name}_brackets'] = s.str.count(r'[\(\)\[\]\{\}]')

        # 7. Отношения пунктуации
        df[f'{name}_exc_ratio'] = df[f'{name}_excl_marks'] / (df[f'{name}_len'] + 1e-5)
        df[f'{name}_quest_ratio'] = df[f'{name}_quest_marks'] / (df[f'{name}_len'] + 1e-5)

        # # 8. Лингвистические паттерны
        # df[f'{name}_avg_sent_len'] = s.apply(lambda x: np.mean([len(sent.split()) for sent in re.split(r'[.!?]', x) if sent.strip()]) if x else 0)
        # df[f'{name}_num_sentences'] = s.apply(lambda x: len([sent for sent in re.split(r'[.!?]', x) if sent.strip()]))
        # df[f'{name}_has_ellipsis'] = s.str.contains(r'\.\.\.').astype(int)
        # df[f'{name}_has_mention'] = s.str.contains(r'@\w+').astype(int)
        # df[f'{name}_has_url'] = s.str.contains(r'https?://|www\.').astype(int)
        # df[f'{name}_has_hashtag'] = s.str.contains(r'#\w+').astype(int)

        # 9. Сложность текста
        df[f'{name}_lexical_richness'] = df[f'{name}_unique_words'] / (df[f'{name}_words'] + 1)
        df[f'{name}_avg_word_complexity'] = s.apply(
            lambda x: np.mean([sum(c in "аеёиоуыэюяaeiouy" for c in w.lower()) / (len(w)+1e-5) for w in x.split()]) if x.split() else 0
        )

text_features(test, 'left', 'title_x', 'content_x')
print('готово1')
text_features(test, 'right', 'title_y', 'content_y')
print('готово2')

готово1
готово2


In [52]:
preds_test = model.predict(test[X.columns])

In [53]:
sub = pd.read_csv('sample_submission.csv')

In [54]:
sub['target'] = preds_test[:, 0]

In [55]:
sub.to_csv('sub_8.csv', index=False)